<a href="https://colab.research.google.com/github/taniokah/Sports_Analysis/blob/master/Player_Tracking_for_ICCP2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import cv2 #　opencvのインポート
import matplotlib.pyplot as plt # matplotlib(描画用)

print(cv2.__version__)

3.4.3


In [0]:
!pip install pytube
!pip install ffmpeg

In [0]:
!rm -rf screen_caps
!mkdir screen_caps

In [0]:
from pytube import YouTube

yt = YouTube("https://youtu.be/l00CwQ6HI_o")


In [0]:
yt.streams.all()
#yt.streams.filter(progressive=False).desc().all()
#yt.streams.get_by_itag(313)
#yt.streams.first().download()
yt.streams.get_by_itag(313).download()

'/content/20190224 鹿屋体育大学 vs 鹿児島高専 1st.webm'

In [0]:
import sys
import cv2
import numpy as np

#動画ファイルを読み込む
#file_name = u"20190217 徳島大学医学部 vs 吉野クラブ 2nd No2.mp4"
#file_name = u"20190224 鹿屋体育大学 vs 鹿児島高専 1st.mp4"
file_name = u"/content/20190224 鹿屋体育大学 vs 鹿児島高専 1st.webm"
cap = cv2.VideoCapture(file_name)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(str(width) + ', ' + str(height))
fps = cap.get(cv2.CAP_PROP_FPS)

#(width, height) = (1280, 720)
print(str(width) + ', ' + str(height))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter('T20190224_1st.mp4', fourcc, fps, (width, height), True)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('frame_count = ' + str(frame_count))

frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
print('frame_rate = ' + str(frame_rate))

#while True:
for i in range(frame_count):
	#_, frame = cap.read()
	ret, frame = cap.read()
	if not ret:
		break

	pts1 = np.float32([[1110, 486], [2700, 486], [-90, 1080], [3870, 1080]])
	pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
	matrix = cv2.getPerspectiveTransform(pts1, pts2)
	result = cv2.warpPerspective(frame, matrix, (width, height))
  
	#cv2.circle(frame, (1110, 486), 5, (0, 0, 255), -1)
	#cv2.circle(frame, (2700, 486), 5, (0, 255, 0), -1)
	#cv2.circle(frame, (-90, 1080), 5, (255, 0, 0), -1)
	#cv2.circle(frame, (3870, 1080), 5, (0, 255, 255), -1)

	#cv2.imshow("Frame", frame)
	#cv2.imshow("Perspective transformation", result)

	writer.write(result)

	key = cv2.waitKey(1)
	if key == 27:
		break

	sys.stdout.write("\r%d" % i)
	sys.stdout.flush()

cv2.destroyAllWindows()
writer.release()
cap.release()
print('done')


3840, 2160
3840, 2160


In [0]:
import sys
import cv2
import numpy as np

cap = cv2.VideoCapture('20190224 鹿屋体育大学 vs 鹿児島高専 1st.webm')
#cap = cv2.VideoCapture('T4K20190511_2st.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(str(width) + ', ' + str(height))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter('T4KT20190224_1st.mp4', fourcc, fps, (width, height), True)

tracker = cv2.MultiTracker_create()

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('frame_count = ' + str(frame_count))

frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
print('frame_rate = ' + str(frame_rate))

shift_time = 180
cap.set(0, shift_time * 1000)

skip = 1000
ret, frame = cap.read()
if not ret:
    print('Failed to read video')
    exit()

cv2.namedWindow('tracking', cv2.WINDOW_KEEPRATIO | cv2.WINDOW_NORMAL)
bbox1 = cv2.selectROI('tracking', frame)

#while True:
for i in range(frame_count):
	index = i + 1
	#_, frame = cap.read()
	ret, frame = cap.read()
	if not ret:
		break

	if i == 0:
		ret = tracker.add(cv2.TrackerMIL_create(), frame, bbox1)
		#bbox2 = cv2.selectROI('tracking', frame)
		#bbox3 = cv2.selectROI('tracking', frame)

	#ok = tracker.add(cv2.TrackerMIL_create(), image, bbox1)
	#ok = tracker.add(cv2.TrackerMIL_create(), image, bbox2)
	#ok = tracker.add(cv2.TrackerMIL_create(), image, bbox3)

	ret, boxes = tracker.update(frame)

	for newbox in boxes:
		p1 = (int(newbox[0]), int(newbox[1]))
		p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
		cv2.rectangle(frame, p1, p2, (200,0,0))

	if index < skip:
		cv2.imshow('tracking', frame)

	#cv2.namedWindow('Frame', cv2.WINDOW_KEEPRATIO | cv2.WINDOW_NORMAL)
	#cv2.imshow("Frame", frame)
	#cv2.namedWindow('Perspective', cv2.WINDOW_KEEPRATIO | cv2.WINDOW_NORMAL)
	#cv2.imshow("Perspective", result)

	writer.write(frame)

	key = cv2.waitKey(1)
	if key == 27:
		break

	sys.stdout.write("\r%d" % index)
	sys.stdout.flush()

cv2.destroyAllWindows()
writer.release()
cap.release()
